In [7]:
import pandas as pd
import pulp

In [8]:
# Cell produces a dict where the key value represents a specific week at a specific facility eg."2023-01-01-F001"
# The value is a tuple that holds the list of demands for a specific product over that week along with the facility id and capacity
optimazation_args_dict = {}
df_demand = pd.read_csv("../../demand.csv")
dr_facilities = pd.read_csv("../../facilities.csv")
tmp = {}

for _, row in dr_facilities.iterrows():
    tmp[row["facility_id"]] = row["throughput_capacity"]

for _, row in df_demand.iterrows():
    key_value = row['date'] + '-' + row['facility_id']
    if key_value not in optimazation_args_dict:
        # Creating first key entry for a specified week and associated faciltiy along with demand list and facility number and capacity
        optimazation_args_dict[key_value] = ([row['demand']], row['facility_id'] ,tmp[row['facility_id']])
    else:
        optimazation_args_dict[key_value][0].append(row['demand'])
print(optimazation_args_dict)

{'2023-01-01-F001': ([156, 524, 44, 167, 147, 54, 61, 113, 218, 100, 91, 752, 57, 62, 89, 132, 98, 209, 68, 82], 'F001', 11389), '2023-01-08-F001': ([224, 732, 74, 232, 137, 83, 80, 176, 197, 189, 83, 758, 158, 89, 109, 117, 95, 192, 69, 116], 'F001', 11389), '2023-01-15-F001': ([179, 829, 74, 232, 102, 97, 75, 169, 254, 190, 100, 877, 242, 85, 102, 129, 106, 199, 72, 94], 'F001', 11389), '2023-01-22-F001': ([167, 951, 95, 186, 136, 70, 95, 143, 277, 155, 72, 667, 210, 111, 109, 185, 106, 258, 121, 161], 'F001', 11389), '2023-01-29-F001': ([159, 750, 85, 212, 142, 80, 90, 136, 198, 177, 90, 832, 182, 111, 94, 116, 125, 209, 91, 141], 'F001', 11389), '2023-02-05-F001': ([175, 780, 93, 164, 110, 69, 80, 161, 204, 144, 83, 751, 166, 66, 66, 116, 109, 233, 78, 143], 'F001', 11389), '2023-02-12-F001': ([138, 845, 70, 223, 131, 61, 65, 197, 175, 119, 65, 773, 157, 78, 66, 121, 101, 247, 108, 136], 'F001', 11389), '2023-02-19-F001': ([102, 845, 92, 135, 98, 61, 36, 141, 141, 121, 65, 453, 109

In [9]:
import pulp

d = [156, 524, 44, 167, 147, 54, 61, 113, 218, 100, 91, 752, 57, 62, 89, 132, 98, 209, 68, 82] # demands
v = [5,4,7,9,2,5,6,8,3,11,1,4,7,2,9,3,7,1,5,8]
W = 1500

V = [di * vi for di, vi in zip(d, v)]  # [12, 2, 6, 15]
n = len(d)

model = pulp.LpProblem("Knapsack", pulp.LpMaximize)
x = [pulp.LpVariable(f"x_{i}", cat="Binary") for i in range(n)]

# Objective: maximize total value
model += pulp.lpSum(V[i] * x[i] for i in range(n))

# Capacity constraint
model += pulp.lpSum(d[i] * x[i] for i in range(n)) <= W

model.solve(pulp.PULP_CBC_CMD(msg=False))

chosen = [i for i in range(n) if pulp.value(x[i]) == 1]
best_value = pulp.value(model.objective)
total_weight = sum(d[i] for i in chosen)

print("Chosen products:", chosen)    
print("Total value:", best_value)     
print("Total weight:", total_weight)  

Chosen products: [0, 1, 2, 3, 6, 7, 9, 12, 14, 16, 19]
Total value: 9599.0
Total weight: 1491
